# $\S7.$ Logistic Regression

**Author**: [Gilyoung Cheong](https://www.linkedin.com/in/gycheong/)

**References**
* ["The Elements of Statistical Learning" by Hastie, Tibshirani, and Friedman](https://hastie.su.domains/ElemStatLearn/)

### General Concept of Logistic Regression

Generally speaking, a "logistic regression" is an application of any regression for a classification problem. The idea is very simple: suppose that we are training a model $(\phi_{\boldsymbol{\beta}} : \mathbb{R}^m \rightarrow \mathbb{R})_{\boldsymbol{\beta} \in \mathcal{P}}$, for a continuous outcome. Here $\mathcal{P}$ is the parameter space, and given $n$ input datasets

* $(x_{11}, x_{12}, \dots, x_{1m})$,
* $(x_{21}, x_{22}, \dots, x_{2m})$,

$\hspace{2cm}\vdots$

* $(x_{n1}, x_{n2}, \dots, x_{nm})$,

in $\mathbb{R}^n$ and $n$ output datasets $y_1, y_2 \dots, y_n$ in $\mathbb{R}$, training the model means choosing $\boldsymbol{\beta} \in \mathcal{P}$ such that 

$$\sum_{i=1}^n L(\phi_{\boldsymbol{\beta}}(x_{i1}, x_{i2}, \dots, x_{im}), y_i)$$ 

is minimized, where we denoted by $L : \mathbb{R} \times \mathbb{R} \rightarrow \mathbb{R}_{\geq 0}$ a loss function. (In particular, training of the model depends on the choice of the loss function.)

Then composing this model with the **sigmoid function** $\sigma : \mathbb{R} \rightarrow [0, 1]$ defined by
$$\sigma(t) := \frac{1}{1 + e^{-t}}$$

gives us a model $(\sigma \circ \phi_{\boldsymbol{t}} : (\mathbb{R}^n)^{m} \rightarrow [0,1])_{\boldsymbol{t} \in \mathcal{P}}$ suitable for predicting the probability distribution of a categorical outcome. Of course, when we train the categorical classifier model, we need to train $\sigma \circ \phi_{\boldsymbol{\beta}}$ not $\phi_{\boldsymbol{\beta}}$.

### Training Logistic Regression: Maximum Likelihood Estimation (MLE)

MLE is a method of estimating the parameters of an assumed probability distribution, given some observed data. That is, we maximize a likelihood function so that the observed data is most probable under the assumed statistical model. For example, the ordinary least square estimator for a linear regression model maximizes the likelihood when the random errors are assumed to have normal distributions with the same variance. (We are not going over this particular example of MLE in this notebook.)

For the logistic regression of a given continuous model $(\phi_{\boldsymbol{\beta}} : \mathbb{R}^m \rightarrow \mathbb{R})_{\boldsymbol{\beta} \in \mathcal{P}}$, for each input dataset $(x_{i1}, \dots, x_{im})$ (which corresponds to $1 \leq i \leq n$), we have

$$\hat{P}_{i,\boldsymbol{\beta}} := \sigma(\phi_{\boldsymbol{\beta}}(x_{i1}, \dots, x_{im})) = \frac{1}{1 + e^{-\phi_{\boldsymbol{\beta}}(x_{i1}, \dots, x_{im})}} \in (0, 1).$$

Equivalently, this can be written as
$$\log\left(\frac{\hat{P}_{i,\boldsymbol{\beta}}}{1 - \hat{P}_{i,\boldsymbol{\beta}}}\right) = \phi_{\boldsymbol{\beta}}(x_{i1}, \dots, x_{im}).$$

Now, our outcome datasets, say $z_1, \dots, z_n$ are taking values in $\{0, 1\}$, so if $P_i$ is the probability that the outcome $z_i$ is equal to $1$ conditional on the occurence of the input dataset $x_{i1}, x_{i2}, \dots, x_{im}$, then, we must have

$$\mathbb{P}(z_i | x_{i1}, x_{i2}, \dots, x_{im}) = P_i^{z_i}(1 - P_i)^{1-z_i}$$

The training for the logistic regression is to use the following belief/principle:

**Belief/Principle**. We believe that  $\hat{P}_{i,\boldsymbol{\beta}} \approx P_i$ for $1 \leq i \leq n$ and that

* the approximation is at best when we choose $\boldsymbol{\beta} \in \mathcal{P}$ so that the quantity

$$\prod_{i=1}^n \hat{P}_{i,\boldsymbol{\beta}}^{z_i}(1 - \hat{P}_{i, \boldsymbol{\beta}})^{1-z_i},$$

which corresponds to

$$\prod_{i=1}^n \mathbb{P}(z_i | x_{i1}, x_{i2}, \dots, x_{im})$$

is maximized. For computational convenience, we maximize the log of the quantity above:

$$\begin{align*}
\sum_{i=1}^n [z_i\log(\hat{P}_{i,\boldsymbol{\beta}}) + (1-z_i)\log(1 - \hat{P}_{i,\boldsymbol{\beta}})] &= \sum_{i=1}^n [-z_i\log(1 + e^{-\phi_{\boldsymbol{\beta}}(x_{i1}, \dots, x_{im})}) + (1-z_i)(\log(e^{-\phi_{\boldsymbol{\beta}}(x_{i1}, \dots, x_{im})}) - \log(1 + e^{-\phi_{\boldsymbol{\beta}}(x_{i1}, \dots, x_{im})}))] \\
&= \sum_{i=1}^n [(z_i - 1)\phi_{\boldsymbol{\beta}}(x_{i1}, \dots, x_{im}) - \log(1 + e^{-\phi_{\boldsymbol{\beta}}(x_{i1}, \dots, x_{im})})]
\end{align*}$$

by choosing $\boldsymbol{\beta} \in \mathcal{P}$.


**Remark**. Although one can feed any regression model into the sigmoid function to create a logistic version of such regression model, but it seems that when we discuss a logistic regression, it is assumed that the regression is a logistic linear ridge regression, as we can see in the documentation of the [LogisticRegression from scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). Since any polynomial regression is a linear regression on the polynomial features of the input data, such a library can be directly used when we consider a logistic polynomial (ridge) regression.

### Logistic Regression (Logistic Linear Regression with Ridge Regularization)

Recall that a ridge linear regression with penality parameter $\lambda \geq 0$ considers a model $(\phi_{\boldsymbol{\beta}} : \mathbb{R}^m \rightarrow \mathbb{R})_{\boldsymbol{\beta} \in \mathbb{R}^{m+1}}$ such that for each input dataset $(x_{i1},x_{i2}, \dots, x_{in})$ data we have

$$\phi_{\boldsymbol{\beta}}(\boldsymbol{x}_1, \dots, \boldsymbol{x}_m) = \beta_0 + \beta_1 x_{i1} + \beta_2 x_{i2} + \cdots + \beta_m x_{im},$$

where we wrote $\boldsymbol{\beta} = (\beta_0, \beta_1, \dots, \beta_m)$.

For the training the given input datasets and the training output data $\boldsymbol{y} = (y_1, \dots, y_n)$ for the *continuous outcome*, writing

$$X = \begin{bmatrix}
1 & x_{11} & x_{12} & \cdots & x_{1m} \\
1 & x_{21} & x_{22} & \cdots & x_{2m} \\
\vdots & \vdots & \vdots & \cdots & \vdots \\
1 & x_{n1} & x_{n2} & \cdots & x_{nm}
\end{bmatrix}$$

the training lets us choose

$$\boldsymbol{\beta} = \boldsymbol{\beta}_{\lambda} = (X^{T}X + \lambda I_{m+1})^{-1}X^{T}\boldsymbol{y}$$

for the ridge regression; when $\lambda > 0$, such an inverse always exists. This is NOT $\boldsymbol{\beta}$ that we choose for the training of logistic regreassion. (We use the MLE described above instead.)